<a href="https://colab.research.google.com/github/qahtanaa/OnSubGroupFairness/blob/main/ProfCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install aif360

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.1/214.1 kB 2.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from aif360.metrics import utils

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


In [43]:
##########

import pandas as pd
import numpy as np

# Create a small DataFrame
np.random.seed(42)
n = 21
data = {
    'Gender': np.random.choice(['F', 'M'], size=n),
    'Race': np.random.choice(['B', 'W'], size=n),
    'NumKids': np.random.choice(range(5), size=n),
    'Height': np.random.uniform(150, 190, size=n),
    'ShirtColor': np.random.choice(['Red', 'Blue', 'Green'], size=n),
    'Age': np.random.randint(18, 65, size=n),
    'Accepted': np.random.choice([1, 2], size=n)
}

df = pd.DataFrame(data)

# Display the DataFrame
#print(df)

#############introduce missing values in the df and delete those rows containing missing values

# Introduce missing values randomly
missing_fraction = 0.1  # Adjust this fraction based on the desired percentage of missing values

# Randomly select cells and set their values to NaN
mask = np.random.rand(*df.shape) < missing_fraction
df[mask] = np.nan

# Display the DataFrame with missing values
#print(df)


initial_rows = len(df)
df = df.dropna()
removed_rows = initial_rows - len(df)

if removed_rows > 0:
    print(f"Detected {removed_rows} rows with missing values. Removed them.")
else:
    print("No missing values detected.")

#print(df)

##############change the label in 0 and 1
label = 'Accepted'
unfav = 1
fav = 2
number_label_values = df[label].nunique()
if number_label_values == 2:
  print(f"The '{label}' column has only two unique values.")
  df[label].replace([unfav, fav], [0, 1], inplace=True)
else:
  print(f"The '{label}' column does not have exactly two unique values, as it should.")

#print(df)

sensitive = ['Gender', 'Race']
group_combinations = pd.MultiIndex.from_product([df[sensitive].unique() for sensitive in sensitive] + [df[label].unique()], names=sensitive + [label])
#print(group_combinations)
print(list(enumerate(group_combinations)))
# Create a mapping between group combinations and their corresponding numbers
group_mapping = {group: idx for idx, group in enumerate(group_combinations)}

# Apply the mapping to create a new column in the DataFrame
df['group'] = pd.MultiIndex.from_frame(df[sensitive + [label]]).map(group_mapping)

#for i in len(group_combinations[1]):
print(df)


Detected 7 rows with missing values. Removed them.
The 'Accepted' column has only two unique values.
[(0, ('F', 'B', 1.0)), (1, ('F', 'B', 0.0)), (2, ('F', 'W', 1.0)), (3, ('F', 'W', 0.0)), (4, ('M', 'B', 1.0)), (5, ('M', 'B', 0.0)), (6, ('M', 'W', 1.0)), (7, ('M', 'W', 0.0))]
   Gender Race  NumKids      Height ShirtColor   Age  Accepted  group
0       F    B      1.0  159.235753       Blue  34.0       1.0      0
1       M    W      3.0  159.641019        Red  25.0       1.0      6
2       F    W      3.0  177.330541        Red  52.0       0.0      3
3       F    W      2.0  174.399866       Blue  52.0       0.0      3
6       F    W      0.0  165.642424       Blue  59.0       0.0      3
9       M    B      2.0  167.006235        Red  45.0       0.0      5
10      F    B      4.0  158.317667        Red  24.0       0.0      1
12      F    W      1.0  151.252532        Red  25.0       1.0      2
13      F    W      3.0  183.691391        Red  29.0       0.0      3
14      M    B      0.

In [41]:
df

,Gender,Race,NumKids,Height,ShirtColor,Age,Accepted,group
0,F,B,1.0,159.235753,Blue,34.0,1.0,0
1,M,W,3.0,159.641019,Red,25.0,1.0,6
2,F,W,3.0,177.330541,Red,52.0,0.0,3
3,F,W,2.0,174.399866,Blue,52.0,0.0,3
6,F,W,0.0,165.642424,Blue,59.0,0.0,3
9,M,B,2.0,167.006235,Red,45.0,0.0,5
10,F,B,4.0,158.317667,Red,24.0,0.0,1
12,F,W,1.0,151.252532,Red,25.0,1.0,2
13,F,W,3.0,183.691391,Red,29.0,0.0,3
14,M,B,0.0,167.990165,Red,51.0,0.0,5


In [6]:
# Professor's code
def get_att_details(att):
    avg_len = 0
    max_len = 0
    num_int = 0
    num_float = 0
    num_text = 0
    thresh = 0.99
    details = dict()
    dtype_dict = dict()
    dtype_dict["typeInt"] = dict()
    dtype_dict["typeFloat"] = dict()
    dtype_dict["typeText"] = dict()
    dtype = ""
    no_non_empty = 0
    att_no_null_values = att.dropna()
    idxA = att_no_null_values.index.tolist()
    no_non_empty = len(att_no_null_values)
    min_len = len(att_no_null_values[idxA[0]])
    for ix in idxA:
        # find min, max, average length of the values
        v = att_no_null_values[ix]
        lv = len(str(v))
        avg_len += lv
        max_len = max(max_len, lv)
        min_len = min(min_len, lv)

        # check the types of the values
        try:
            val = int(v)
            num_int += 1
            if lv in dtype_dict["typeInt"]:
                dtype_dict["typeInt"][lv] += 1
            else:
                dtype_dict["typeInt"][lv] = 1
        except ValueError:
            try:
                val = float(v)
                num_float += 1
                if lv in dtype_dict["typeFloat"]:
                    dtype_dict["typeFloat"][lv] += 1
                else:
                    dtype_dict["typeFloat"][lv] = 1
            except ValueError:
                num_text += 1
                if lv in dtype_dict["typeText"]:
                    dtype_dict["typeText"][lv] += 1
                else:
                    dtype_dict["typeText"][lv] = 1
    if no_non_empty > 0:
        avg_len /= no_non_empty
    details["max_len"] = max_len
    details["avg_len"] = avg_len
    details["min_len"] = min_len
    # print(num_int, no_non_empty, thresh, num_int / no_non_empty)
    if num_int / no_non_empty > thresh:
        lens = sorted(dtype_dict["typeInt"].items(), key=lambda kv: kv[1], reverse=True)
        aa, bb = lens[0]
        if (bb / num_int > thresh) and (min_len >= 2):
            dtype = "code"
        else:
            dtype = "integer"
        # print(lens)
    if num_float / no_non_empty > thresh:
        dtype = "float"
    if dtype == "":
        lens = sorted(dtype_dict["typeText"].items(), key=lambda kv: kv[1], reverse=True)
        if len(lens) > 0:
            aa, bb = lens[0]
            if bb / num_text > thresh:
                dtype = "code"
            else:
                dtype = "text"
            details["lens"] = lens
        else:
            details["lens"] = []
    details["dtype"] = dtype

    return details


def get_df_details(df):
    df_details = dict()
    df_details.clear()
    for att_id in range(len(df.columns)):
        att = df[df.columns[att_id]]
        details = get_att_details(att)
        df_details[att_id] = details
        df_details[att_id]["att_name"] = df.columns[att_id]
    return df_details


def find_cand_atts(df_dets):
  cand_atts = []
  for cc in range(len(cand_atts)):
        cand_atts.remove(cand_atts[0])
  for att_id in df_dets.keys():
    if (df_dets[att_id]["dtype"] == "integer") or (df_dets[att_id]["dtype"] == "float"):
      continue
    cand_atts.append(att_id)
  return cand_atts


In [48]:
# using only categorical attributes

df_1 = df[['Gender','Race', 'ShirtColor']]
df_det_1 = get_df_details(df_1)
find_cand_atts(df_det_1)

[0, 1, 2]

In [50]:
#using categorical attributes and a numerical attribute (Age)
df_2 = df[['Gender','Race', 'ShirtColor', 'Age']]
df_det_2 = get_df_details(df_2)
find_cand_atts(df_det_2)

TypeError: object of type 'numpy.float64' has no len()